## Update the restoration BMPs table

1) Go to https://www.fielddoc.org/ and download the DRRF program practices as a geopackage (.gpkg). Place the geopackage in the data folder and update the file path in the 3rd code block. 

2) Run all cells, wait for it to be done.

3) Go back to the WikiSRAT_Demo notebook and run the "Repeat for Restoration Results" code to load cache the microservice outputs for the restoration run. In that Jupyter notebook, do the following:
      * Import packages
      * Run all cells in Section 2.1 
      * Run all cells in Section 5.7


In [1]:
import fiona
import requests
import json
import collections
import matplotlib as plt
import geopandas as gpd
import pandas as pd
import shapely
import geojson
from shapely.ops import unary_union
from shapely.geometry import polygon
import psycopg2
#from shapely.geometry import shape, mapping
#from geojson import MultiPolygon

requests.adapters.DEFAULT_RETRIES = 1

In [2]:
# GET THE DATABASE CONFIG INFORMATION USING A CONFIG FILE. THE FILE IS IN THE GITIGNORE SO WILL REQUIRE BEING SENT

config_file = json.load(open('db_config.json'))
PG_CONFIG = config_file['PG_CONFIG']

_host = PG_CONFIG['host'],
_database = PG_CONFIG['database'],
_user = PG_CONFIG['user'],
_password = PG_CONFIG['password'],
_port = PG_CONFIG['port']

In [3]:
'''
Load the BMP data from FD Geopackage
Download the gpkg and place into the data folder in the repo
'''
bmps = []

with fiona.open('data/drrf_20210927.gpkg') as layer:
    for feature in layer:
        feature['bmp_geometry'] = feature.pop('geometry')
        bmps.append(feature)


In [4]:
# Create connection to database

_PG_Connection = psycopg2.connect(
        host=PG_CONFIG['host'],
        database=PG_CONFIG['database'],
        user=PG_CONFIG['user'],
        password=PG_CONFIG['password'],
        port=PG_CONFIG['port'])

# Create the table to import the fielddoc data into
cur = _PG_Connection.cursor()
rest_table_name = 'fd_drrf_20210913'

drop_table = 'drop table if exists datapolassess.{} CASCADE;'.format(rest_table_name)
create_table = 'create table datapolassess.{}' \
                       '(practice_id int,practice_name varchar(1024),practice_type varchar(1024),practice_description varchar(1024),practice_url varchar(1024),' \
                       'created_on date,modified_on date,creator_name varchar(1024),creator_id int,site_id int,site_name varchar(1024),site_url varchar(1024),' \
                       'project_id int,project_name varchar(1024),project_status varchar(1024),project_url varchar(1024),program_id int,program_name varchar(1024),' \
                       'program_url varchar(1024),tn_reduced_lbs numeric(24,4),tp_reduced_lbs numeric(24,4),tss_reduced_lbs numeric(24,4), ' \
                       'geom_poly geometry(multipolygon,32618), geom_line geometry(multilinestring,32618), geom_pt geometry(multipoint,32618));'.format(rest_table_name)

grant_select = 'grant select on datapolassess.{} to srat_select;'.format(rest_table_name)
add_pkey = 'alter table datapolassess.{} add constraint pk_{} primary key (practice_id);'.format(rest_table_name, rest_table_name)
cur.execute(drop_table)
cur.execute(create_table)
cur.execute(add_pkey)
cur.execute(grant_select)
_PG_Connection.commit()
cur.close()
print('done')

done


In [5]:
print(bmps[0].keys())
print(bmps[0]['type'])
# print(bmps[0]['properties'].keys())
keep_keys = ['practice_id','practice_name','practice_type','practice_description','practice_url','created_on','modified_on'
,'creator_name','creator_id','site_id','site_name','site_url','project_id','project_name','project_status','project_url'
,'program_id','program_name','program_url','pounds_of_total_nitrogen_reduced_modeled_true_6a2cc9cd7d'
,'pounds_of_total_phosphorus_reduced_modeled_true_ef360259c9','pounds_of_total_suspended_solids_reduced_modeled_true_ae55682491']

keep_idx = [0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,48,51,52]
print(bmps[0]['bmp_geometry']['type'])
print(type(bmps[0]['bmp_geometry']))
# dict(bmps[i]['properties'])

# ST_MULTI(ST_TRANSFORM(ST_SetSRID(ST_GeomFromGeoJSON(),4326),32618))
# geojson.loads(json.dumps(bmps[0]['bmp_geometry']))

dict_keys(['type', 'id', 'properties', 'bmp_geometry'])
Feature
Polygon
<class 'dict'>


In [6]:
cur = _PG_Connection.cursor()

# Insert FD data into the created table
t = 0 # INDEX TO BREAK LOOP IF THERE IS A DB ERROR
a = 0 # COUNTER FOR BMP of BMPS in list
for i in range(0,len(bmps)):
    c = 0 # COUNTER IN CASE WE WANT TO USE 
    values = []
    for k1, v1 in bmps[i].items():
        for k2, v2 in bmps[i]['properties'].items():
            # If you want to add by FD column index number
#             if c in keep_idx:
#                 values.append(str(v2).replace("'", ""))
#             c +=1
            # If you want to add by FD column name
            if k2 in keep_keys:
                values.append(str(v2).replace("'", ""))
    values = [w.replace('None', str(-9999.0)) for w in values]
    bmp_geom = geojson.loads(json.dumps(bmps[i]['bmp_geometry']))
    if bmps[i]['bmp_geometry']['type'] == 'Polygon' or bmps[i]['bmp_geometry']['type'] == 'MultiPolygon':
        try:
            insert_into = 'insert into  datapolassess.{} (practice_id,practice_name,practice_type,practice_description,practice_url,' \
                              'created_on,modified_on,creator_name,creator_id,site_id,site_name,site_url,project_id,project_name,project_status,' \
                              'project_url,program_id,program_name,program_url,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,geom_poly) ' \
                              'values ({},\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',{},{},\'{}\',\'{}\',{},\'{}\',\'{}\',' \
                              '\'{}\',{},\'{}\',\'{}\',{},{},{},ST_MULTI(ST_TRANSFORM(ST_SetSRID(ST_GeomFromGeoJSON(\'{}\'),4326),32618))::geometry(Multipolygon, 32618));'.format(
                rest_table_name,
                values[0], values[1], values[2], values[3], values[4], values[5][:10], values[6][:10], values[7], values[8],
                values[9],values[10], values[11], values[12], values[13], values[14], values[15], values[16], values[17], values[18],
                values[19], values[20], values[21],bmp_geom)
            
            cur.execute(insert_into)
                
        except Exception as e:
            print(e)
            print(insert_into)
            t += 1
    elif bmps[i]['bmp_geometry']['type'] == 'LineString' or bmps[i]['bmp_geometry']['type'] == 'MultiLineString':
        try:
            insert_into = 'insert into  datapolassess.{} (practice_id,practice_name,practice_type,practice_description,practice_url,' \
                              'created_on,modified_on,creator_name,creator_id,site_id,site_name,site_url,project_id,project_name,project_status,' \
                              'project_url,program_id,program_name,program_url,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,geom_line) ' \
                              'values ({},\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',{},{},\'{}\',\'{}\',{},\'{}\',\'{}\',' \
                              '\'{}\',{},\'{}\',\'{}\',{},{},{},ST_MULTI(ST_TRANSFORM(ST_SetSRID(ST_GeomFromGeoJSON(\'{}\'),4326),32618))::geometry(MultiLineString, 32618));'.format(
                rest_table_name,
                values[0], values[1], values[2], values[3], values[4], values[5][:10], values[6][:10], values[7], values[8],
                values[9],values[10], values[11], values[12], values[13], values[14], values[15], values[16], values[17], values[18],
                values[19], values[20], values[21],bmp_geom)
            
            cur.execute(insert_into)
                
        except Exception as e:
            print(e)
            print(insert_into)
            t += 1
    elif bmps[i]['bmp_geometry']['type'] == 'Point' or bmps[i]['bmp_geometry']['type'] == 'MultiPoint':
        try:
            insert_into = 'insert into  datapolassess.{} (practice_id,practice_name,practice_type,practice_description,practice_url,' \
                              'created_on,modified_on,creator_name,creator_id,site_id,site_name,site_url,project_id,project_name,project_status,' \
                              'project_url,program_id,program_name,program_url,tn_reduced_lbs,tp_reduced_lbs,tss_reduced_lbs,geom_pt) ' \
                              'values ({},\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',\'{}\',{},{},\'{}\',\'{}\',{},\'{}\',\'{}\',' \
                              '\'{}\',{},\'{}\',\'{}\',{},{},{},ST_MULTI(ST_TRANSFORM(ST_SetSRID(ST_GeomFromGeoJSON(\'{}\'),4326),32618))::geometry(MultiPoint, 32618));'.format(
                rest_table_name,
                values[0], values[1], values[2], values[3], values[4], values[5][:10], values[6][:10], values[7], values[8],
                values[9],values[10], values[11], values[12], values[13], values[14], values[15], values[16], values[17], values[18],
                values[19], values[20], values[21],bmp_geom)
            
            cur.execute(insert_into)
                
        except Exception as e:
            print(e)
            print(insert_into)
            t += 1
    a += 1
    if t == 1:
        break
            

_PG_Connection.commit()
cur.close()
print('done')

done


In [7]:
cur = _PG_Connection.cursor()
create_fin_geom = 'alter table datapolassess.fd_drrf_20210913 add column geom_fin geometry(multipolygon,32618);' \
                    'update datapolassess.fd_drrf_20210913 set geom_fin = geom_poly::geometry(multipolygon,32618) where geom_poly is not null and geom_fin is null;' \
                    'update datapolassess.fd_drrf_20210913 set geom_fin = st_multi(ST_buffer(geom_line,1))::geometry(multipolygon,32618) where geom_line is not null and geom_fin is null;' \
                    'update datapolassess.fd_drrf_20210913 set geom_fin = st_multi(ST_buffer(geom_pt,1))::geometry(multipolygon,32618) where geom_pt is not null and geom_fin is null;'
cur.execute(create_fin_geom)
_PG_Connection.commit()
cur.close()
print('done')

done


In [8]:
cur = _PG_Connection.cursor()
create_comid_table1 = 'update datapolassess.{} a set geom_fin = ST_CollectionExtract(st_makevalid(geom_fin),3);'.format(rest_table_name)
create_comid_table2 = 'create index restoration_lbsreduced_comid_geom_idx on datapolassess.{} using gist(geom_fin);'.format(rest_table_name)
create_comid_table3 = 'create or replace view datapolassess.restoration_lbsreduced_comid ' \
                        'as ' \
                        'select distinct comid as comid_rest ' \
                        ',sum(tn_reduced_lbs_int/2.20462) over (partition by comid) as tn_reduced_kg ' \
                        ',sum(tp_reduced_lbs_int/2.20462) over (partition by comid) as tp_reduced_kg ' \
                        ',sum(tss_reduced_lbs_int/2.20462) over (partition by comid) as tss_reduced_kg ' \
                        'from ( ' \
                            'select distinct * ' \
                            ',(case when tn_reduced_lbs >= 0 then (tn_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4) as tn_reduced_lbs_int ' \
                            ',(case when tp_reduced_lbs >= 0 then (tp_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4)  as tp_reduced_lbs_int ' \
                            ',(case when tss_reduced_lbs >= 0 then (tss_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4)  as tss_reduced_lbs_int ' \
                            'from ( ' \
                                'select distinct t2.comid, t1.* ' \
                                ',st_multi(st_intersection(t1.geom_fin, t2.catchment))::geometry(multipolygon,32618) as geom_int ' \
                                'from ( ' \
                                    'select distinct * ' \
                                    'from datapolassess.{} ' \
                                    'where project_status in (\'active\', \'complete\') and practice_name not like \'%copy%\' ' \
                                ') as t1 ' \
                                'left join spatial.nhdplus_maregion as t2 ' \
                                'on st_intersects(t1.geom_fin, t2.catchment) ' \
                            ') as t3 ' \
                            'order by comid, practice_id ' \
                        ') as t4 ' \
                        'order by comid;'.format(rest_table_name)
create_comid_table4 = 'drop table if exists datapolassess.cache_restoration_lbsreduced_comid; ' \
                        'create table datapolassess.cache_restoration_lbsreduced_comid ' \
                        'as ' \
                        'select distinct * from datapolassess.restoration_lbsreduced_comid where comid_rest is not null and (tn_reduced_kg + tp_reduced_kg + tss_reduced_kg) > 0.0; ' \
                        'alter table datapolassess.cache_restoration_lbsreduced_comid add constraint pk_cache_restoration_lbsreduced_comid primary key (comid_rest); '

cur.execute(create_comid_table1)
cur.execute(create_comid_table2)
cur.execute(create_comid_table3)
cur.execute(create_comid_table4)
_PG_Connection.commit()
cur.close()
print('done')


done


In [9]:
cur = _PG_Connection.cursor()
get_by_project_view = 'create or replace view datapolassess.restoration_lbsreduced AS ' \
                         ' select distinct comid, practice_id, site_id,practice_type, ' \
                         '(case when tn_reduced_lbs >= 0 then (tn_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4) as tn_reduced_lbs, ' \
                         '(case when tp_reduced_lbs >= 0 then (tp_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4)  as tp_reduced_lbs, ' \
                         '(case when tss_reduced_lbs >= 0 then (tss_reduced_lbs * (st_area(geom_int)/st_area(geom_fin))) else 0.0 end)::numeric(20,4)  as tss_reduced_lbs, ' \
                         'practice_name,case when practice_description like \'-9999.0\' then null else practice_description end as practice_description, ' \
                         'project_name,project_status,creator_name,program_id,program_name,created_on,modified_on,practice_url, project_url,geom_fin as geom ' \
                         'from ( select distinct t2.comid, t1.* ,st_multi(st_intersection(t1.geom_fin, t2.catchment))::geometry(multipolygon,32618) as geom_int ' \
                         'from (select distinct *from datapolassess.fd_drrf_20210913 where project_status in (\'active\', \'complete\') and practice_name not like \'%copy%\') as t1 ' \
                         'left join spatial.nhdplus_maregion as t2 ' \
                         'on st_intersects(t1.geom_fin, t2.catchment)) as t3 ' \
                         'where (tn_reduced_lbs + tn_reduced_lbs + tn_reduced_lbs) > 0.0 ' \
                         'order by comid, practice_id;'

cur.execute(get_by_project_view)
_PG_Connection.commit()
cur.close()

In [10]:
# cur = _PG_Connection.cursor()


# get_non_null = 'create or replace view datapolassess.restoration_lbsreduced AS ' \
#                     ' select * from datapolassess.restoration_lbsreduced_comid where comid_rest is not null'

# _PG_Connection.commit()
# cur.close()

In [11]:
# SJ updated this one 

cur = _PG_Connection.cursor()

get_by_project_table = 'drop table if exists datapolassess.cache_restoration_lbsreduced;' \
                               'create table datapolassess.cache_restoration_lbsreduced ' \
                               'as select distinct * from datapolassess.restoration_lbsreduced ' \
                               'where comid is not null; ' \
                               'alter table datapolassess.cache_restoration_lbsreduced add constraint pk_cache_restoration_lbsreduced primary key (comid, practice_id);'

cur.execute(get_by_project_table)
_PG_Connection.commit()
cur.close()
print('done')

done


## END OF CODE